In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"E:\imbcifar\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|█████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.53it/s]

X_train shape: (29009, 32, 32, 3)
Y_train shape: (29009,)


In [2]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"E:\imbcifar\valid"

# Initialize empty lists for images and labels
X_val = []
Y_val = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_val.append(img_array)
            Y_val.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print(f'X_train shape: {X_val.shape}')
print(f'Y_train shape: {Y_val.shape}')

100%|█████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.11it/s]

X_train shape: (3222, 32, 32, 3)
Y_train shape: (3222,)


In [3]:
import tensorflow as tf
#tf.keras.backend.clear_session()
from tensorflow.keras.utils import to_categorical
YT = to_categorical(Y_train)
VT = to_categorical(Y_val)

In [8]:
Y_val

array([0, 0, 0, ..., 9, 9, 9])

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 10
# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights=None, include_top=False, input_shape=(32, 32, 3))    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
model.fit(X_train,YT, epochs=epochs,batch_size=32, validation_split = .1)

816/816 [==============================] - 37s 36ms/step - loss: 2.0293 - accuracy: 0.3700 - val_loss: 4.2055 - val_accuracy: 0.0000e+00


In [5]:
print(".......................................................................................")
print(".......................................................................................")

.......................................................................................
.......................................................................................


In [10]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(X_val)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_val, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.2436374922408442
F1 Score: 0.1726247538470893


In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [14]:
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 10
model = tensorflow.keras.models.load_model(r"D:\imbcifar_model.h5")

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D(keepdims=False)(x)  # Explicitly specify keepdims
output = Dense(num_classes, activation='softmax')(x)
new_model = tensorflow.keras.models.Model(inputs=model.input, outputs=output) 
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
new_model.fit(X_train,YT, epochs=epochs,batch_size=32, validation_split = .1)

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


816/816 [==============================] - 29s 31ms/step - loss: 1.3689 - accuracy: 0.5985 - val_loss: 3.1298 - val_accuracy: 0.0855


In [15]:
print(".......................................................................................")
print(".......................................................................................")

.......................................................................................
.......................................................................................


In [16]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = new_model.predict(X_val)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_val, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.5238981998758535
F1 Score: 0.46506765778486703


In [ ]:
print(".......................................................................................")
print(".......................................................................................")